<a href="https://colab.research.google.com/github/Ryan-Riggs/myrepo/blob/master/vertices_each_scene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()
import folium

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=YS6AJjPy5HZ5_rOhpCMfTTDVH_Vx63R3sxBC2u5swc0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3QFs3HF1zX1PtEuZ-1Sm4r1tJY_lNAdVODKq3D91s0n3G2viBU7GDv4

Successfully saved authorization token.


In [4]:
grwl_cline = ee.FeatureCollection('users/eeProject/GRWL_summaryStats')
NA = ee.FeatureCollection("users/rriggs/na_sj_using_R_min_100")
fc_3spc = ee.FeatureCollection("users/rriggs/na_sj_using_R_min_100")
x = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_waterClassification_Jones2019.py'
fls = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
flsh = 'users/eeProject/RivWidthCloudPaper:rwc_landsat.py'
fnsLandsat = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
lsFun = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
riverFun = 'users/eeProject/RivWidthCloudPaper:functions_river.py'
gauges = ee.FeatureCollection("users/rriggs/Gauge_points")
gauges_filt = gauges.filter(ee.Filter.gt('GRWL_width', 99))

def switchGeometryLine2Endpoints(f):
        f = f.set({'lineGeometry': f.geometry()})
        l = f.geometry().coordinates()
        g = ee.Geometry.MultiPoint(l, 'EPSG:4326')
        return(f.setGeometry(g))

def switchGeometryEndpoints2Line(f):
        return(f.setGeometry(f.get('lineGeometry')).set('lineGeometry', None))

def feat2fc(f):
        point = ee.Feature(ee.Geometry.Point(f), {})
        return(point)

def pt_limit(f):
        geom = f.geometry().coordinates()
        point = geom.map(feat2fc)
        point2 = ee.FeatureCollection(point)#.randomColumn().filter(ee.Filter.gt('random', 0.99))
        return(point2)



In [5]:
def Mndwi(image):
    return(image.normalizedDifference(['Green', 'Swir1']).rename('mndwi'))

def Mbsrv(image):
    return(image.select(['Green']).add(image.select(['Red'])).rename('mbsrv'))

def Mbsrn(image):
    return(image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn'))

def Ndvi(image):
    return(image.normalizedDifference(['Nir', 'Red']).rename('ndvi'))

def Awesh(image):
    return(image.expression('Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2', {
    'Blue': image.select(['Blue']),
    'Green': image.select(['Green']),
    'mbsrn': Mbsrn(image).select(['mbsrn']),
    'Swir2': image.select(['Swir2'])}))

def Dswe(i):
    mndwi = Mndwi(i)
    mbsrv = Mbsrv(i)
    mbsrn = Mbsrn(i)
    awesh = Awesh(i)
    swir1 = i.select(['Swir1'])
    nir = i.select(['Nir'])
    ndvi = Ndvi(i)
    blue = i.select(['Blue'])
    swir2 = i.select(['Swir2'])

    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = (mndwi.gt(-0.44)
    .And(swir1.lt(900))
    .And(nir.lt(1500))
    .And(ndvi.lt(0.7)))
    t5 = (mndwi.gt(-0.5)
    .And(blue.lt(1000))
    .And(swir1.lt(3000))
    .And(swir2.lt(1000))
    .And(nir.lt(2500)))

    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    noWater = (t.eq(0)
    .Or(t.eq(1))
    .Or(t.eq(10))
    .Or(t.eq(100))
    .Or(t.eq(1000)))
    hWater = (t.eq(1111)
    .Or(t.eq(10111))
    .Or(t.eq(11011))
    .Or(t.eq(11101))
    .Or(t.eq(11110))
    .Or(t.eq(11111)))
    mWater = (t.eq(111)
    .Or(t.eq(1011))
    .Or(t.eq(1101))
    .Or(t.eq(1110))
    .Or(t.eq(10011))
    .Or(t.eq(10101))
    .Or(t.eq(10110))
    .Or(t.eq(11001))
    .Or(t.eq(11010))
    .Or(t.eq(11100)))
    pWetland = t.eq(11000)
    lWater = (t.eq(11)
    .Or(t.eq(101))
    .Or(t.eq(110))
    .Or(t.eq(1001))
    .Or(t.eq(1010))
    .Or(t.eq(1100))
    .Or(t.eq(10000))
    .Or(t.eq(10001))
    .Or(t.eq(10010))
    .Or(t.eq(10100)))

    iDswe = (noWater.multiply(0)
    .add(hWater.multiply(1))
    .add(mWater.multiply(2))
    .add(pWetland.multiply(3))
    .add(lWater.multiply(4)))

    return(iDswe.rename(['dswe']))

def ClassifyWaterJones2019(img):
    dswe = Dswe(img)
    waterMask = dswe.eq(1).Or(dswe.eq(2)).rename(['waterMask'])
    return(waterMask)

def switchGeometryLine2Endpoints(f):
        f = f.set({'lineGeometry': f.geometry()})
        l = f.geometry().coordinates()
        g = ee.Geometry.MultiPoint(l, 'EPSG:4326')
        return(f.setGeometry(g))

def switchGeometryEndpoints2Line(f):
        return(f.setGeometry(f.get('lineGeometry')).set('lineGeometry', None))

def GetCenterline(clDataset, bound):
    # // filter the GRWL centerline based on area of interest
    cl = clDataset.filterBounds(bound)
    return(cl)

def ExtractChannel(image):
    # // extract the channel water bodies from the water mask, based on connectivity to the reference centerline.
    connectedToCl = (image.Not().cumulativeCost(
        source = ee.Image().toByte().paint(grwl_cline, 1).And(image), #// only use the centerline that overlaps with the water mask
        maxDistance = 4000,
        geodeticDistance = False).eq(0))

    channel = image.updateMask(connectedToCl).unmask(0).updateMask(image.gte(0)).rename(['channelMask'])
    return(channel)


def RemoveIsland(channel):
    # /* fill in island as water if the size (number of pixels) of the island is smaller than FILL_SIZE */
    fill = channel.Not().selfMask().connectedPixelCount(333).lt(333)
    river = channel.where(fill, ee.Image(1)).rename(['riverMask'])
    return(river)

def ExtractRiver(imgIn, clData, maxDist, minIslandRemoval):
    waterMask = imgIn.select(['waterMask'])
    bound = waterMask.geometry()
    cl = GetCenterline(clData, bound)
    channelMask = ExtractChannel(waterMask, cl, maxDist)
    riverMask = RemoveIsland(channelMask, minIslandRemoval)
    return(imgIn.addBands(channelMask).addBands(riverMask))

def widths(image):
  width = (image.eq(1).reduceRegions(
  collection = filt, 
  reducer= ee.Reducer.mean(),
  ))
  flags = (image.reduceRegions(
  collection= width.map(switchGeometryLine2Endpoints), 
  reducer= ee.Reducer.max(),
))
  return(flags)

def filt_lines (f):
  return f.set('geo_type', f.geometry().type())

def maximum_no_of_tasks(MaxNActive, waitingPeriod):
	"""maintain a maximum number of active tasks
	"""
	import time
	import ee
	ee.Initialize()

	time.sleep(10)
	## initialize submitting jobs
	ts = list(ee.batch.Task.list())

	NActive = 0
	for task in ts:
		if ('RUNNING' in str(task) or 'READY' in str(task)):
			NActive += 1
	## wait if the number of current active tasks reach the maximum number
	## defined in MaxNActive
	while (NActive >= MaxNActive):
		time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
		ts = list(ee.batch.Task.list())
		NActive = 0
		for task in ts:
			if ('RUNNING' in str(task) or 'READY' in str(task)):
				NActive += 1
	return()
def buffer_zone (f):
  return f.buffer(5000)
def  ftr_coll (f):
  return ee.FeatureCollection(f)

def distance_fun (f):
  l = f.geometry().distance(vertices.geometry())
  d = f.set('distance', l)
  return(d)

def distance_fun_gauges (f):
  l = f.geometry().distance(gauges_filt.geometry())
  d = f.set('distance', l)
  return(d)  

In [ ]:
a = [18,
17,
16,
15,
14,
26,
25,
24,
23,
22,
21,
20,
19,
18,
44,
43,
42,
41,
40,
39,
38,
37,
34,
33,
32,
31,
30,
29,
28,
27,
24,
23,
22,
21,
10,
9,
8,
51,
50,
49,
48,
47,
46,
45,
40,
39,
38,
37,
36,
35,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
20,
19,
17,
16,
14,
13,
7,
18,
17,
22,
21,
18,
17,
16,
15,
10,
36,
35,
34,
33,
32,
31,
30,
28,
27,
26,
25,
24,
23,
22,
21,
20,
19,
16,
12,
7,
48,
47,
46,
45,
44,
43,
42,
41,
40,
39,
37,
36,
35,
34,
33,
32,
31,
29,
28,
27,
26,
25,
24,
23,
22,
21,
15,
14,
13,
12,
11,
10,
55,
54,
53,
48,
47,
46,
45,
44,
43,
28,
27,
26,
25,
24,
23,
20,
11,
10,
9,
20,
19,
18,
17,
16,
15,
14,
13,
12,
11,
20,
19,
18,
17,
16,
15,
14,
6,
43,
42,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
26,
25,
23,
22,
21,
19,
18,
9,
50,
49,
48,
47,
46,
45,
35,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
19,
18,
17,
16,
12,
6,
48,
47,
27,
25,
24,
16,
15,
23,
22,
18,
17,
16,
15,
13,
12,
19,
18,
17,
16,
14,
13,
12,
11,
9,
8,
7,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
18,
17,
16,
15,
14,
12,
47,
46,
45,
44,
43,
42,
41,
40,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
13,
11,
10,
9,
55,
54,
47,
46,
45,
44,
43,
42,
41,
33,
32,
31,
30,
26,
25,
24,
23,
19,
18,
10,
9,
10,
9,
8,
7,
21,
20,
19,
18,
17,
13,
12,
24,
23,
22,
21,
19,
15,
14,
41,
40,
39,
38,
37,
36,
35,
33,
32,
31,
30,
29,
28,
27,
26,
25,
17,
14,
13,
10,
49,
48,
47,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
20,
19,
12,
11,
48,
47,
24,
23,
22,
15,
14,
13,
16,
15,
15,
14,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
20,
19,
17,
45,
44,
43,
42,
41,
40,
39,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
20,
19,
12,
10,
9,
54,
53,
52,
51,
47,
46,
45,
44,
43,
42,
41,
40,
30,
29,
26,
24,
23,
22,
21,
20,
19,
17,
16,
15,
14,
9,
8,
16,
9,
8,
7,
20,
19,
18,
17,
16,
12,
11,
22,
21,
20,
19,
18,
17,
16,
15,
14,
13,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
25,
24,
23,
22,
21,
20,
19,
18,
14,
13,
8,
49,
48,
47,
46,
33,
32,
30,
29,
28,
26,
25,
24,
23,
22,
21,
16,
15,
13,
12,
11,
48,
47,
46,
45,
30,
29,
28,
27,
26,
25,
24,
23,
13,
12,
11,
17,
16,
15,
14,
13,
26,
25,
24,
22,
17,
16,
15,
14,
13,
12,
44,
43,
42,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
23,
22,
21,
20,
19,
18,
16,
15,
13,
11,
10,
9,
52,
51,
50,
45,
44,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
20,
19,
18,
16,
15,
14,
13,
18,
17,
8,
22,
21,
20,
18,
17,
16,
15,
14,
13,
11,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
49,
48,
47,
46,
45,
44,
43,
42,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
13,
12,
11,
10,
55,
54,
53,
48,
47,
46,
45,
25,
24,
21,
20,
12,
11,
10,
11,
10,
9,
8,
18,
17,
16,
15,
14,
13,
12,
11,
23,
19,
18,
17,
16,
15,
43,
42,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
27,
26,
25,
24,
23,
22,
21,
20,
18,
11,
51,
50,
49,
48,
47,
46,
40,
39,
38,
37,
36,
29,
28,
26,
25,
24,
23,
22,
21,
20,
19,
18,
16,
14,
13,
48,
27,
26,
25,
23,
22,
18,
16,
18,
17,
16,
15,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
18,
17,
16,
15,
13,
12,
11,
10,
48,
47,
46,
45,
44,
43,
42,
41,
38,
37,
36,
35,
34,
33,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
20,
14,
13,
12,
11,
55,
54,
48,
47,
46,
45,
44,
43,
42,
30,
29,
28,
27,
26,
25,
24,
23,
19,
18,
10,
9,
11,
10,
13,
12,
11,
24,
23,
22,
19,
18,
17,
16,
15,
14,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
23,
22,
21,
17,
11,
50,
49,
48,
47,
46,
45,
40,
30,
29,
28,
27,
26,
25,
23,
22,
21,
20,
19,
18,
11,
15,
14,
15,
13,
12,
10,
17,
16,
15,
12,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
20,
19,
18,
15,
14,
13,
11,
46,
45,
44,
43,
42,
41,
40,
39,
38,
37,
36,
35,
25,
24,
23,
13,
11,
10,
54,
47,
46,
45,
44,
43,
42,
41,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
21,
17,
16,
15,
14,
9,
21,
20,
19,
18,
17,
16,
15,
14,
13,
12,
23,
22,
21,
20,
19,
18,
17,
14,
10,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
10,
49,
48,
47,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
27,
26,
25,
24,
23,
22,
21,
18,
17,
14,
13,
12,
11,
48,
47,
29,
28,
27,
26,
25,
24,
23,
22,
14,
13,
12,
12,
10,
18,
17,
16,
15,
14,
13,
27,
26,
24,
17,
16,
13,
42,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
24,
23,
22,
21,
20,
19,
16,
15,
11,
53,
52,
51,
50,
49,
45,
44,
43,
42,
41,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
21,
20,
19,
17,
16,
15,
14,
20,
19,
18,
17,
16,
12,
11,
18,
17,
16,
15,
14,
13,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
23,
22,
21,
18,
17,
16,
14,
13,
49,
48,
47,
46,
45,
40,
39,
38,
37,
36,
35,
34,
33,
32,
31,
30,
29,
28,
27,
26,
25,
24,
23,
22,
56,
55,
54,
53,
48,
47,
46,
45,
29,
28,
27,
26,
25,
24,
21,
13,
12,
11,
10,
24,
23,
22,
21,
13,
28,
27,
39,
38,
13,
26,
14,
15,
23,
22,
21,
20,
19,
13,
39,
35,
28,
27,
21,
16,
15,
14,
20,
18,
17,
16,
34,
24,
23,
22,
40,
39,
38,
18,
17,
29,
28,
27,
26,
25,
23,
18,
17,
14,
13,
12,
18,
17,
16,
13,
16,
14,
13,
12,
38,
37,
36,
35,
34,
33,
50,
37,
36,
35,
34,
33,
32,
31,
28,
27,
25,
15,
14,
21,
14,
13,
28,
27,
17,
16,
37,
36,
35,
34,
31,
27,
22,
21,
13,
18,
23,
21,
20,
19,
18,
27,
25,
24,
49,
21,
28,
27,
19,
14,
13,
12,
21,
20,
19,
18,
17,
16,
15,
37,
23,
22,
21,
20,
19,
16,
44,
29,
28,
27,
26,
21,
20,
14,
45,
32,
31,
30,
27,
21,
19,
17,
15,
13,
19,
14,
13,
22,
21,
20,
19,
40,
39,
16,
15,
32,
31,
20,
19,
18,
16,
15,
14,
21,
20,
25,
39,
38,
37,
36,
35,
34,
33,
32,
31,
13,
48,
47,
25,
24,
23,
23,
22,
18,
17,
14,
13,
17,
47,
31,
30,
29,
28,
27,
22,
21,
20,
19,
16,
15,
20,
19,
18,
14,
13,
16,
15,
25,
24,
22,
21,
20,
19,
17,
17,
16,
14,
25,
21,
20,
19,
18,
15,
44,
43,
18,
18,
17,
21,
15,
21,
20,
19,
24,
20,
19,
22,
20,
16,
15,
17,
16,
15,
14,
36,
35,
26,
19,
22,
21,
18,
28,
27,
26,
25,
19,
18,
17,
29,
38,
32,
31,
30,
29,
16,
15,
37,
36,
34,
33,
15,
17,
29,
46,
16,
15,
18,
40,
26,
39,
38,
17,
15,
17,
16,
56,
28,
19,
16,
35,
34,
33,
32,
31,
53,
27,
26,
16,
17,
27,
26,
34,
33,
32,
51,
16,
28,
46,
18,
22,
26,
25,
17,
22,
18,
21,
17,
30,
23,
34,
19,
18,
21,
19,
20,
21,
20,
18,
22,
40,
26,
17,
18,
17]
b = [66,
66,
66,
66,
66,
50,
50,
50,
50,
50,
50,
50,
50,
50,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
34,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
18,
2,
2,
75,
75,
75,
75,
75,
75,
75,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
27,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
11,
68,
68,
68,
68,
68,
68,
68,
68,
68,
68,
52,
52,
52,
52,
52,
52,
52,
52,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
36,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
20,
4,
4,
4,
4,
4,
4,
4,
77,
77,
77,
77,
77,
77,
77,
77,
61,
61,
61,
61,
61,
61,
61,
61,
61,
61,
61,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
29,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
13,
230,
230,
230,
230,
70,
70,
70,
70,
70,
70,
70,
54,
54,
54,
54,
54,
54,
54,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
22,
6,
6,
6,
6,
6,
6,
6,
6,
79,
79,
63,
63,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
31,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
232,
232,
232,
232,
72,
72,
72,
72,
72,
72,
72,
56,
56,
56,
56,
56,
56,
56,
56,
56,
56,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
40,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
24,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
65,
65,
65,
65,
65,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
33,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
17,
1,
1,
1,
74,
74,
74,
74,
74,
74,
74,
58,
58,
58,
42,
42,
42,
42,
42,
42,
42,
42,
42,
42,
42,
42,
42,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
26,
10,
10,
10,
10,
10,
10,
10,
10,
10,
10,
10,
10,
10,
10,
227,
227,
227,
227,
67,
67,
67,
67,
67,
67,
67,
67,
51,
51,
51,
51,
51,
51,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
35,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
19,
3,
3,
3,
3,
76,
76,
76,
76,
60,
60,
60,
60,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
28,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
12,
229,
229,
69,
69,
69,
53,
53,
53,
53,
53,
53,
53,
53,
53,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
37,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
5,
5,
78,
78,
78,
78,
62,
62,
62,
62,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
14,
71,
71,
71,
71,
71,
71,
71,
71,
71,
71,
55,
55,
55,
55,
55,
55,
55,
55,
55,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
23,
7,
7,
7,
7,
7,
7,
7,
7,
7,
7,
7,
7,
7,
80,
80,
64,
64,
64,
64,
64,
64,
48,
48,
48,
48,
48,
48,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
32,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
16,
73,
73,
73,
73,
73,
73,
73,
57,
57,
57,
57,
57,
57,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
25,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
52,
52,
52,
52,
52,
36,
36,
20,
20,
20,
4,
77,
61,
45,
45,
45,
45,
45,
45,
29,
13,
13,
13,
13,
70,
70,
70,
54,
54,
54,
54,
38,
38,
38,
38,
22,
22,
22,
22,
22,
6,
6,
6,
6,
6,
79,
79,
79,
79,
79,
79,
63,
63,
63,
63,
47,
47,
47,
47,
31,
31,
31,
31,
31,
31,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
15,
232,
232,
72,
72,
72,
40,
40,
40,
40,
24,
24,
24,
24,
24,
24,
8,
8,
81,
65,
49,
49,
49,
49,
49,
33,
33,
33,
17,
17,
1,
1,
74,
74,
74,
74,
58,
58,
58,
58,
58,
58,
58,
42,
42,
42,
42,
42,
42,
42,
10,
10,
10,
10,
10,
51,
51,
51,
19,
19,
19,
19,
19,
76,
76,
76,
76,
76,
60,
60,
60,
44,
44,
44,
44,
28,
28,
28,
28,
12,
12,
69,
69,
69,
69,
69,
69,
53,
53,
37,
21,
21,
21,
21,
21,
21,
21,
21,
21,
21,
5,
5,
5,
5,
5,
78,
78,
78,
78,
62,
62,
46,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
30,
14,
14,
14,
231,
231,
55,
55,
39,
39,
39,
39,
39,
39,
39,
80,
80,
80,
48,
48,
48,
48,
48,
48,
32,
32,
32,
233,
233,
73,
73,
57,
57,
57,
41,
41,
41,
9,
9,
82,
82,
50,
50,
50,
50,
34,
34,
34,
34,
18,
18,
18,
2,
2,
2,
2,
75,
59,
59,
43,
27,
11,
11,
11,
11,
36,
36,
20,
20,
20,
20,
20,
4,
13,
31,
31,
31,
15,
40,
40,
24,
24,
24,
81,
33,
26,
10,
35,
35,
35,
19,
19,
19,
28,
28,
12,
37,
37,
37,
21,
5,
5,
30,
30,
30,
14,
39,
23,
7,
80,
48,
32,
32,
32,
16,
16,
25,
25,
9,
9,
18,
59,
43,
11,
11,
36,
38,
38,
232,
72,
24,
33,
17,
3,
3]

In [6]:
a=[22,
26,
25,
17,
22,
18,
21,
17,
30,
23,
34,
19,
18,
21,
19,
20,
21,
20,
18,
22,
40,
26,
17,
18,
17]
b=[48,
32,
32,
32,
16,
16,
25,
25,
9,
9,
18,
59,
43,
11,
11,
36,
38,
38,
232,
72,
24,
33,
17,
3,
3]

In [8]:
#fc_3spc = fc_3spc.filterBounds(rc_roi)
LT5_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5',    'B7',    'B6'];
LE7_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5',    'B7',    'B6_VCID_1'];
LC8_BANDS = ['B2',   'B3',    'B4',  'B5',  'B6',    'B7',    'B10'];
STD_NAMES = ['Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'Temp'];

#// load Landsat 5,7,8 collections:
#// TODO(GHA): combine 5, 7, and 8 collections:
LT5 = ee.ImageCollection('LANDSAT/LT5_L1T_TOA').select(LT5_BANDS, STD_NAMES);
LT5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA').select(LT5_BANDS, STD_NAMES); 
##LE7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
##.select(LE7_BANDS, STD_NAMES);
LC8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').select(LC8_BANDS, STD_NAMES);

collection = LC8.merge(LT5)
#filtered = collection.filterDate('1985-01-01', '1985-01-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 100).filterMetadata('WRS_PATH', 'equals', 39).filterMetadata('WRS_ROW', 'equals', 23)

In [ ]:
grwl_cline_filt = grwl_cline.filterBounds(filtered.first().geometry())
max = 99
grwl_cline_max = grwl_cline_filt.filter(ee.Filter.gt('width_mean', max)).filter(ee.Filter.eq('lakeFlag', 0))
grwl_cline_max_value = grwl_cline_max.aggregate_max('width_mean')
grwl_cline_max_filt = grwl_cline_max.filter(ee.Filter.eq('width_mean', grwl_cline_max_value))
grwl_cline_max_filt = grwl_cline_max_filt.map(switchGeometryLine2Endpoints)
vertices = grwl_cline_max_filt.map(pt_limit)
vertices = vertices.flatten()
vertices = vertices.filterBounds(filtered.first().geometry())
vertices = vertices.randomColumn()
vertices = vertices.limit(5, 'random')
vertices_1 = vertices.map(buffer_zone)


In [ ]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [9]:
def buffer_zone (f):
  return(f.buffer(5000))

def maximum_no_of_tasks(MaxNActive, waitingPeriod):
	"""maintain a maximum number of active tasks
	"""
	import time
	import ee
	ee.Initialize()

	time.sleep(10)
	## initialize submitting jobs
	ts = list(ee.batch.Task.list())

	NActive = 0
	for task in ts:
		if ('RUNNING' in str(task) or 'READY' in str(task)):
			NActive += 1
	## wait if the number of current active tasks reach the maximum number
	## defined in MaxNActive
	while (NActive >= MaxNActive):
		time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
		ts = list(ee.batch.Task.list())
		NActive = 0
		for task in ts:
			if ('RUNNING' in str(task) or 'READY' in str(task)):
				NActive += 1
	return()
 
def ind (f):
 buff1 = ee.FeatureCollection(f).map(buffer_zone)
 fi1 = buff1.map(ftr_coll)
 filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
 fff = filt1.map(distance_fun)
 filt = fff.limit(5, 'distance')
 return(filt)

def ind_gauge (f):
 buff1 = ee.FeatureCollection(f).map(buffer_zone)
 fi1 = buff1.map(ftr_coll)
 filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
 fff = filt1.map(distance_fun_gauges)
 filt = fff.limit(5, 'distance')
 return(filt)

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=1)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:

#from folium import plugins
import folium
Map = geemap.Map(center=[40,-100], zoom=1)
Map
#map
#map.add_ee_layer(vertices_1, {'palette':['red']}, 'vertices')
#map.add_child(folium.LayerControl())
#plugins.Fullscreen().add_to(map)
#display(map)
#Map.addLayer(vertices, {'color': 'green'}, 'vertices')
Map.add_ee_layer(gauges)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:
##for loop to create a giant map of all locations in the output. 
for i in range(len(a)):
  filtered = collection.filterDate('1985-01-01', '1985-01-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 100).filterMetadata('WRS_PATH', 'equals', b[i]).filterMetadata('WRS_ROW', 'equals', a[i])
  grwl_cline_filt = grwl_cline.filterBounds(filtered.first().geometry())
  max = 99
  grwl_cline_max = grwl_cline_filt.filter(ee.Filter.gt('width_mean', max)).filter(ee.Filter.eq('lakeFlag', 0))
  grwl_cline_max_value = grwl_cline_max.aggregate_max('width_mean')
  grwl_cline_max_filt = grwl_cline_max.filter(ee.Filter.eq('width_mean', grwl_cline_max_value))
  grwl_cline_max_filt = grwl_cline_max_filt.map(switchGeometryLine2Endpoints)
  vertices = grwl_cline_max_filt.map(pt_limit)
  vertices = vertices.flatten()
  vertices = vertices.filterBounds(filtered.first().geometry())
  vertices = vertices.randomColumn()
  vertices = vertices.limit(5, 'random')
  vertices_1 = vertices.map(buffer_zone)
  #Map.add_ee_layer(vertices_1,{'palette':['red']}, 'vertices')
  #Map.add_ee_layer(vertices,{'palette':['blue']}, 'vertices')
  task = (ee.batch.Export.table.toDrive(
  collection = vertices_1,
  description = 'widths_' + '_' + str(a[i]),
  folder = 'Random_points',
  fileNamePrefix = 'Random_points' + '_' + str(a[i]) + str(b[i]),
  fileFormat = 'shp'
  ))
  task.start()
#print(output.first())
  print('task', i, 'has started')
  maximum_no_of_tasks(3, 60)


In [ ]:
##for loop to determine random points in each landsat scene from largest river and calculate widths. 
for i in range(len(a)):
  filtered = collection.filterDate('1984-01-01', '2020-12-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 10).filterMetadata('WRS_PATH', 'equals', b[i]).filterMetadata('WRS_ROW', 'equals', a[i])
  grwl_cline_filt = grwl_cline.filterBounds(filtered.first().geometry())
  max = 99
  grwl_cline_max = grwl_cline_filt.filter(ee.Filter.gt('width_mean', max)).filter(ee.Filter.eq('lakeFlag', 0))
  grwl_cline_max_value = grwl_cline_max.aggregate_max('width_mean')
  grwl_cline_max_filt = grwl_cline_max.filter(ee.Filter.eq('width_mean', grwl_cline_max_value))
  grwl_cline_max_filt = grwl_cline_max_filt.map(switchGeometryLine2Endpoints)
  vertices = grwl_cline_max_filt.map(pt_limit)
  vertices = vertices.flatten()
  vertices = vertices.filterBounds(filtered.first().geometry())
  vertices = vertices.randomColumn()
  vertices = vertices.limit(5, 'random')
  vertices_1 = vertices.map(buffer_zone)
  filt = vertices_1.map(ind)
  filt = filt.flatten()
  waterMask = filtered.map(ClassifyWaterJones2019)
  GetCenterline(grwl_cline, filt)
  riverMask = waterMask.map(ExtractChannel)
  #riverMask_noislands = riverMask.map(RemoveIsland)
  w = riverMask.map(widths)
  tbl = w.flatten()
  sel_xsections = tbl.filter(ee.Filter.gt('mean', 0)).filter(ee.Filter.eq('max', 0))
  output = sel_xsections.select(
    ['ID', 'mean', 'date', 'width_m', 'lat_dd', 'lon_dd', 'ID_2', 'max','COMID'])
      # export widths

  task = (ee.batch.Export.table.toDrive(
  collection = output,
  description = 'PR' + str(a[i]) + str(b[i]),
  folder = 'Scene_test',
  fileNamePrefix = 'PR' + str(a[i]) + str(b[i]),
  fileFormat = 'csv'
  ))

  task.start()
#print(output.first())
  print('task', i, 'has started')
  print('task has started')
  maximum_no_of_tasks(3, 60)

In [ ]:
Map.add_layer_control()
Map

In [ ]:
Map

In [ ]:
gauges_filt1 = gauges_filt.filterBounds(filtered.first().geometry())
print(gauges_filt1.getInfo())
filt = gauges_filt1.map(ind_gauge)
filt = filt.flatten()
waterMask = filtered.map(ClassifyWaterJones2019)
GetCenterline(grwl_cline, filt)
riverMask = waterMask.map(ExtractChannel)
  #riverMask_noislands = riverMask.map(RemoveIsland)
w = riverMask.map(widths)
tbl = w.flatten()
sel_xsections = tbl.filter(ee.Filter.gt('mean', 0)).filter(ee.Filter.eq('max', 0))
output = sel_xsections.select(
  ['ID', 'mean', 'date', 'width_m', 'lat_dd', 'lon_dd', 'ID_2', 'max','COMID'])

In [ ]:
##for loop to determine widths at each gauge per scene
for i in range(len(a)):
  filtered = collection.filterDate('2010-01-01', '2020-12-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 10).filterMetadata('WRS_PATH', 'equals', b[i]).filterMetadata('WRS_ROW', 'equals', a[i])
  gauges_filt1 = gauges_filt.filterBounds(filtered.geometry())
  filt = gauges_filt1.map(ind_gauge)
  filt = filt.flatten()
  waterMask = filtered.map(ClassifyWaterJones2019)
  GetCenterline(grwl_cline, filt)
  riverMask = waterMask.map(ExtractChannel)
  #riverMask_noislands = riverMask.map(RemoveIsland)
  w = riverMask.map(widths)
  tbl = w.flatten()
  sel_xsections = tbl.filter(ee.Filter.gt('mean', 0)).filter(ee.Filter.eq('max', 0))
  output = sel_xsections.select(
    ['ID', 'mean', 'date', 'width_m', 'lat_dd', 'lon_dd', 'ID_2', 'max','COMID'])
      # export widths

  task = (ee.batch.Export.table.toDrive(
  collection = output,
  description = 'PR' + str(a[i]) + str(b[i]),
  folder = 'Gauges_per_scene',
  fileNamePrefix = 'PR' + str(a[i]) + str(b[i]),
  fileFormat = 'csv'
  ))

  task.start()
#print(output.first())
  print('task', i, 'has started')
  #print('task has started')
  maximum_no_of_tasks(3, 60)